In [9]:
import numpy as np
from QAM_EncoderDecoder import * 
import commpy

N = 1024
prefix_no = 32
Fs = 441000
filename='input.png'
bits=file_to_bitstr(filename)
symbols = encode_bitstr2symbols(bits)
frames = symbol_to_OFDMframes(symbols,N,prefix_no)
frames = np.ndarray.flatten(np.array(frames))


about to do encode_bitstr2symbols encoding
encode_bitstr2symbols encoding finished
about to do symbol_to_OFDMframes encoding
symbol_to_OFDMframes encoding finished


Generate Baseband Signal with root raised cosine pulse with scikit-commpy package

``commpy.filters.rrcosfilter``

https://commpy.readthedocs.io/en/latest/generated/commpy.filters.rrcosfilter.html


- N (int) – Length of the filter in samples.
- alpha (float) – Roll off factor (Valid values are [0, 1]).
- Ts (float) – Symbol period in seconds.
- Fs (float) – Sampling Rate in Hz.

In [10]:
alpha = 0.5
Ts = 48 / Fs
pulse = np.array(commpy.filters.rrcosfilter(N, alpha, Ts, Fs)[1][100:])
pulse.shape


(924,)

In [11]:
print(pulse.shape, np.shape(frames))
baseband_signal = np.convolve(frames, pulse) 
baseband_signal

(924,) (671616,)


array([ 1.00639876e-04+0.j,  2.01911848e-04+0.j,  4.12421248e-04+0.j, ...,
       -1.87659287e-04+0.j, -1.30558501e-04+0.j, -6.41214674e-05+0.j])

In [12]:
baseband_signal = np.real(baseband_signal)
baseband_signal

array([ 1.00639876e-04,  2.01911848e-04,  4.12421248e-04, ...,
       -1.87659287e-04, -1.30558501e-04, -6.41214674e-05])

In [13]:
len(baseband_signal)

672539

In [14]:
from scipy.io.wavfile import write, read

fs = 44100  # Sample rate
# Save the numpy as wav audio file
write('ofdmbaseband_channel_input.wav', fs, baseband_signal)

### Receiver: Low pass + Matched filtering 